# Prepping Data

In [46]:
# importing libraries
import requests
import re
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import time

In [47]:
# loading dataframe 
df = pd.read_csv('https://raw.githubusercontent.com/michaelseman/Portfolio/main/PythonML_NFL_Draft/BioPlusConfBMI.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7576 entries, 0 to 7575
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               7576 non-null   int64  
 1   round              7576 non-null   int64  
 2   draft_pick         7576 non-null   int64  
 3   draft_team         7576 non-null   object 
 4   player             7576 non-null   object 
 5   player_url         7100 non-null   object 
 6   position           7576 non-null   object 
 7   age                7095 non-null   float64
 8   college            7560 non-null   object 
 9   college_stats_url  5748 non-null   object 
 10  pic_url            4122 non-null   object 
 11  bday               7095 non-null   object 
 12  hs                 6808 non-null   object 
 13  height             7088 non-null   object 
 14  weight             7088 non-null   float64
 15  height_decimal     7088 non-null   float64
 16  birthplace_town    6713 

In [48]:
df['position'].unique()

array(['DE', 'CB', 'T', 'OL', 'WR', 'DT', 'S', 'G', 'QB', 'LB', 'DB',
       'RB', 'TE', 'OLB', 'K', 'P', 'DL', 'C', 'FB', 'LS', 'ILB'],
      dtype=object)

In [49]:
# offense_positions = ['QB','RB','TE','FB','WR']
offense_positions = ['QB']

## selecting positions for dataframe

In [50]:
# getting only offensive positions (not Oline)
df_off = df[df['position'].isin(offense_positions)]

In [51]:
df_off.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 19 to 7573
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               351 non-null    int64  
 1   round              351 non-null    int64  
 2   draft_pick         351 non-null    int64  
 3   draft_team         351 non-null    object 
 4   player             351 non-null    object 
 5   player_url         320 non-null    object 
 6   position           351 non-null    object 
 7   age                320 non-null    float64
 8   college            350 non-null    object 
 9   college_stats_url  317 non-null    object 
 10  pic_url            221 non-null    object 
 11  bday               320 non-null    object 
 12  hs                 286 non-null    object 
 13  height             319 non-null    object 
 14  weight             319 non-null    float64
 15  height_decimal     319 non-null    float64
 16  birthplace_town    293 n

## dropping all columns without a college stats url

In [52]:
# dropping all rows where college_stats_url is null
df_qb = df_off.dropna(subset=['college_stats_url'])

In [53]:
df_qb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317 entries, 19 to 7573
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               317 non-null    int64  
 1   round              317 non-null    int64  
 2   draft_pick         317 non-null    int64  
 3   draft_team         317 non-null    object 
 4   player             317 non-null    object 
 5   player_url         297 non-null    object 
 6   position           317 non-null    object 
 7   age                297 non-null    float64
 8   college            317 non-null    object 
 9   college_stats_url  317 non-null    object 
 10  pic_url            212 non-null    object 
 11  bday               297 non-null    object 
 12  hs                 273 non-null    object 
 13  height             297 non-null    object 
 14  weight             297 non-null    float64
 15  height_decimal     297 non-null    float64
 16  birthplace_town    279 n

## reseting dataframe index

In [54]:
df_qb = df_qb.reset_index(drop=True)

# Define columns

In [1]:
def_columns = ['year', 'school', 'conference', 'class', 'position', 'games', 'solo_tackles', 'assisted_tackles', 'total_tackles', 'tackles_loss', 'sacks', 'interceptions', 'int_yards', 'int_yards_per_int', 'int_td', 'passes_defended', 'fumbles_recovered', 'fumble_yards', 'fumble_td', 'forced_fumbles']
punt_columns = ['year','school','conference','class','position','games','punt_returns','punt_ret_yds','punt_ret_avg','punt_td','kick_returns','kick_ret_yds','kick_ret_avg','kick_ret_td']
pass_columns = ['year','school','conference','class','position','games','completions','attempts','comp_pct','passing_yds','yds_per_att','adj_yds_per_att','passing_td','interceptions','pass_eff_rate']
rush_columns = ['year','school','conference','class','position','games','rush_attempts','rush_yds','rush_avg','rush_td','receptions','rec_yds','rec_avg','rec_td','r_total_plays','r_total_yds','r_total_avg','r_total_td']
rec_columns = ['year','school','conference','class','position','games','receptions','rec_yds','rec_avg','rec_td','rush_attempts','rush_yds','rush_avg','rush_td','total_plays','total_yds','total_avg','total_td']

In [51]:
# making a small df to test loops with
df_test = pd.DataFrame(df_qb.iloc[0]).T
df_test = df_test.reset_index(drop=True)

In [38]:
df_test

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi
19,2022,1,20,PIT,Kenny Pickett,https://www.pro-football-reference.com/players...,QB,24.0,Pittsburgh,http://www.sports-reference.com/cfb/players/ke...,...,6/6/1998,"Ocean Twp.,NJ",6'3,220.0,6.25,Oakhurst,"NJ,",NJ,ACC,27.5


# Function: take_url_return_page

In [35]:
def take_url_return_page(url):
  # this function takes a url and returns the soup content for it
  r = requests.get(url)
  page = BeautifulSoup(r.content, 'html.parser')
  return page

# Function: take_page_return_tables

In [36]:
def take_page_return_tables(page):
  # getting main table which should always be passing
  pass_table = page.find('table')
  
  # we need to check it the main table is actually the passing table, so we get the id
  try:
    table_id = pass_table.get('id')
  except AttributeError:
    table_id = None
  
  if table_id == 'passing':
    # passing table id is the normal outcome.  We then look for rushing table
    all_rushing = page.find(id='all_rushing')

    if all_rushing is None:
      # just in case they have no rushing table
      return pass_table, 0
    else:
      # how to handle finding a secondary commented out table like rushing
      rushing = all_rushing(text=lambda text: isinstance(text, Comment))[0]
      rush_soup = BeautifulSoup(rushing, 'html.parser')
      rush_table = rush_soup.find('table', {'id':'rushing'})
      return pass_table, rush_table

  elif table_id is None:
    # this outcome means the player has no stats, we set rush_table to none, pass_table is already none in this case
    rush_table = None
    return pass_table, rush_table
  else:
    # this is if the main table is not passing... if that is the case the main table for a QB would be rushing
    rush_table = pass_table
    all_passing = page.find(id='all_passing')
    passing = all_passing(text=lambda text: isinstance(text, Comment))[0]
    pass_soup = BeautifulSoup(passing, 'html.parser')
    pass_table = pass_soup.find('table', {'id':'pass'})
    return pass_table, rush_table


# Function: take_table_return_stats

In [37]:
def take_table_return_stats(table, columns):
  # this function takes the stats table and returns a dataframe of the stats
  
  # define the DataFrame
  df = pd.DataFrame(columns=columns)  

  # finds all rows in the table
  try:
    rows = table.find_all('tr')
  
    

    # this counter will be used along with total_games to get the total games played in the final row of the dataframe and to index the dataframe to exit the loop.
    row_counter = 0
    total_games = 0

    # loop over each row in the table, skipping over header rows (0,1)
    for row in rows[2:]:
      # creating a list where we will store the stats for each row.  We will then append this list to our df
      stat_list = []
      
      if row.find('th').text == 'Career':
        stat_list.append(row.find('th').text)
      else:
      # this part is for the first rows of our table.  these rows contain the year in an 'a' tag
        try: 
          stat_list.append(row.find('a').text)
        except AttributeError:
          # after year values, we find the text 'Career' inside a th tag.
          try:
            stat_list.append(row.find('th').text)
          # if neither 'a' nor 'th' tags are present
          except AttributeError:
            break
          
      # cells will contain all the data inside our table td tags
      cells = row.find_all('td')
      
      # counter to be used for total games
      cell_counter=0
        
      # loop for pulling all the relevant stats from cells
      for cell in cells:
        # when the cell count is 4, that cell contains the games played... we want to total this for use in the last row
        if cell_counter == 4:
          games = cell.text
          # try/except for catch nulls error
          try:
            games = int(games)
            total_games += games
          except ValueError:
            total_games += 0
          stat_list.append(games)
          cell_counter +=1
        
        # this is for all the other cells
        else:
          stat_list.append(cell.text)
          cell_counter +=1
      
      df = df.append(pd.DataFrame([stat_list], columns=columns))
      # this is our break to stop the issue when a player has multiple colleges and they are tallied AFTER career
      if df.iloc[row_counter,0]=='Career':
        for i in range(2,5):
          df.iloc[row_counter,i]= df.iloc[row_counter-1,i]
        df.iloc[row_counter,5] = total_games
        break
      row_counter+=1
  except AttributeError:
    df = pd.DataFrame()
  return df

# Function: the_loop

In [38]:
def the_loop2(original_df):
  for i in range(len(original_df)):
    # get the page and tables for this row
    page = take_url_return_page(original_df.iloc[i,9])
    pass_table, rush_table = take_page_return_tables(page)

    # get the passing stats and update the column names
    stat_df = take_table_return_stats(pass_table, pass_columns)
    if stat_df.empty:
      print('This player has NO pass stats')
    else:
      row_to_add = stat_df[stat_df['year'] == 'Career']
      prefix = str(row_to_add.loc[0, 'year']) + '_'
      updated_columns = [prefix + col for col in pass_columns]
      row_to_add.columns = updated_columns


      # update the original row with the new Career row
      original_df.loc[i, updated_columns] = row_to_add.iloc[0]
      print(original_df.loc[i])
      
      subset_df = stat_df.iloc[:-1, :]
      row_to_add = None

      # loop to find the last season where they played 6 or more games.  We will save this as their 'best' season stats in the df
      for j in range(len(subset_df)-1, -1, -1):
          row = subset_df.iloc[j]
          games_str = row['games']
          if games_str:
              games = int(games_str)
          else:
              games = 0 # or some other default value
          if games >= 6:
              row_to_add = row
              break

      if row_to_add is None:
          row_to_add = subset_df.iloc[-1]

      # add the string 'Best_' to all the column names in row_to_add
      prefix = 'Best_'
      updated_columns = [prefix + col for col in row_to_add.index]
      row_to_add.columns = updated_columns

      original_df.loc[i, updated_columns] = row_to_add.values
    time.sleep(7)
  return original_df

# Testing the loop

In [31]:
type(df_test)

pandas.core.series.Series

In [42]:
df_qb = df_qb.query('year >= 1995 and year <= 1995')
df_qb = df_qb.reset_index(drop=True)

In [55]:
df_results = the_loop(df_qb)

<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is de

year                                                                   2022
round                                                                     1
draft_pick                                                               20
draft_team                                                              PIT
player                                                        Kenny Pickett
player_url                https://www.pro-football-reference.com/players...
position                                                                 QB
age                                                                    24.0
college                                                          Pittsburgh
college_stats_url         http://www.sports-reference.com/cfb/players/ke...
pic_url                   https://www.pro-football-reference.com/req/201...
bday                                                               6/6/1998
hs                                                            Ocean Twp.,NJ
height      

<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is de

year                              2022
round                                3
draft_pick                          74
draft_team                         ATL
player                  Desmond Ridder
                             ...      
Career_rec_td                      NaN
Career_r_total_plays               NaN
Career_r+total_yds                 NaN
Career_r_total_avg                 NaN
Career_r_total_td                  NaN
Name: 1, Length: 63, dtype: object
year                              2022
round                                3
draft_pick                          74
draft_team                         ATL
player                  Desmond Ridder
                             ...      
Career_rec_td                        1
Career_r_total_plays               503
Career_r+total_yds                2188
Career_r_total_avg                 4.3
Career_r_total_td                   29
Name: 1, Length: 63, dtype: object


<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is de

...

year                             1993
round                               8
draft_pick                        216
draft_team                        PIT
player                  Alex Van Pelt
                            ...      
Career_rec_td                     NaN
Career_r_total_plays              NaN
Career_r+total_yds                NaN
Career_r_total_avg                NaN
Career_r_total_td                 NaN
Name: 314, Length: 63, dtype: object
year                             1993
round                               8
draft_pick                        216
draft_team                        PIT
player                  Alex Van Pelt
                            ...      
Career_rec_td                        
Career_r_total_plays              107
Career_r+total_yds                -99
Career_r_total_avg               -0.9
Career_r_total_td                   6
Name: 314, Length: 63, dtype: object


<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is de

year                           1993
round                             8
draft_pick                      219
draft_team                      SFO
player                  Elvis Grbac
                           ...     
Career_rec_td                   NaN
Career_r_total_plays            NaN
Career_r+total_yds              NaN
Career_r_total_avg              NaN
Career_r_total_td               NaN
Name: 315, Length: 63, dtype: object
year                           1993
round                             8
draft_pick                      219
draft_team                      SFO
player                  Elvis Grbac
                           ...     
Career_rec_td                      
Career_r_total_plays             80
Career_r+total_yds             -239
Career_r_total_avg             -3.0
Career_r_total_td                 1
Name: 315, Length: 63, dtype: object


<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-37-508ec87a5abd>:61: FutureWarning: The frame.append method is de

year                           1993
round                             8
draft_pick                      222
draft_team                      LAC
player                  Trent Green
                           ...     
Career_rec_td                   NaN
Career_r_total_plays            NaN
Career_r+total_yds              NaN
Career_r_total_avg              NaN
Career_r_total_td               NaN
Name: 316, Length: 63, dtype: object
year                           1993
round                             8
draft_pick                      222
draft_team                      LAC
player                  Trent Green
                           ...     
Career_rec_td                      
Career_r_total_plays            305
Career_r+total_yds              516
Career_r_total_avg              1.7
Career_r_total_td                20
Name: 316, Length: 63, dtype: object


In [56]:
df_results.head()

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Career_rush_avg,Career_rush_td,Career_receptions,Career_rec_yds,Career_rec_avg,Career_rec_td,Career_r_total_plays,Career_r+total_yds,Career_r_total_avg,Career_r_total_td
0,2022,1,20,PIT,Kenny Pickett,https://www.pro-football-reference.com/players...,QB,24.0,Pittsburgh,http://www.sports-reference.com/cfb/players/ke...,...,1.9,20,2,-9,-4.5,1,418,800,1.9,21
1,2022,3,74,ATL,Desmond Ridder,https://www.pro-football-reference.com/players...,QB,23.0,Cincinnati,http://www.sports-reference.com/cfb/players/de...,...,4.4,28,2,8,4.0,1,503,2188,4.3,29
2,2022,3,86,TEN,Malik Willis,https://www.pro-football-reference.com/players...,QB,23.0,Liberty,http://www.sports-reference.com/cfb/players/ma...,...,5.8,29,1,13,13.0,0,367,2144,5.8,29
3,2022,3,94,CAR,Matt Corral,https://www.pro-football-reference.com/players...,QB,23.0,Mississippi,http://www.sports-reference.com/cfb/players/ma...,...,4.0,18,1,19,19.0,0,335,1357,4.1,18
4,2022,4,137,NWE,Bailey Zappe,https://www.pro-football-reference.com/players...,QB,23.0,Western Kentucky,http://www.sports-reference.com/cfb/players/ba...,...,0.3,3,0,0,,0,51,17,0.3,3


In [57]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    317 non-null    int64  
 1   round                   317 non-null    int64  
 2   draft_pick              317 non-null    int64  
 3   draft_team              317 non-null    object 
 4   player                  317 non-null    object 
 5   player_url              297 non-null    object 
 6   position                317 non-null    object 
 7   age                     297 non-null    float64
 8   college                 317 non-null    object 
 9   college_stats_url       317 non-null    object 
 10  pic_url                 212 non-null    object 
 11  bday                    297 non-null    object 
 12  hs                      273 non-null    object 
 13  height                  297 non-null    object 
 14  weight                  297 non-null    fl

In [59]:
df_results.to_csv('qb_stats93_22.csv')

# Updating the_loop to include rushing career stats

In [39]:
def the_loop(original_df):
  for i in range(len(original_df)):
    # get the page and tables for this row
    page = take_url_return_page(original_df.iloc[i,9])
    pass_table, rush_table = take_page_return_tables(page)

    # get the passing stats and update the column names
    stat_df = take_table_return_stats(pass_table, pass_columns)
    if stat_df.empty:
      print('This player has NO pass stats')
    else:
      row_to_add = stat_df[stat_df['year'] == 'Career']
      prefix = str(row_to_add.loc[0, 'year']) + '_'
      updated_columns = [prefix + col for col in pass_columns]
      row_to_add.columns = updated_columns


      # update the original row with the new Career row
      original_df.loc[i, updated_columns] = row_to_add.iloc[0]
      print(original_df.loc[i])
      
      subset_df = stat_df.iloc[:-1, :]
      row_to_add = None

      # loop to find the last season where they played 6 or more games.  We will save this as their 'best' season stats in the df
      for j in range(len(subset_df)-1, -1, -1):
          row = subset_df.iloc[j]
          games_str = row['games']
          if games_str:
              games = int(games_str)
          else:
              games = 0 # or some other default value
          if games >= 6:
              row_to_add = row
              break

      if row_to_add is None:
          row_to_add = subset_df.iloc[-1]

      # add the string 'Best_' to all the column names in row_to_add
      prefix = 'Best_'
      updated_columns = [prefix + col for col in row_to_add.index]
      row_to_add.columns = updated_columns
      original_df.loc[i, updated_columns] = row_to_add.values
      
    row_to_add = None
    rush_df = take_table_return_stats(rush_table, rush_columns)
    if rush_df.empty:
      print('This player has NO RUSHING stats')
    else:
      row_to_add = rush_df[rush_df['year'] == 'Career']
      prefix = str(row_to_add.loc[0, 'year']) + '_'
      updated_columns = [prefix + col for col in rush_columns]
      row_to_add.columns = updated_columns


      # update the original row with the new Career row
      original_df.loc[i, updated_columns] = row_to_add.iloc[0]
      print(original_df.loc[i])
    time.sleep(7)
  return original_df